potential visualizations:

group by album, top tracks vs others
correlation between rankings and audio features
audio features by song
map by country - popularity, top tracks

In [156]:
import spotipy
from config import CLIENT_ID, CLIENT_SECRET, PLAY_LIST, USER, DIRECT_URI
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [161]:
spotify = spotipy.Spotify(requests_timeout=10, client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                                              client_secret=CLIENT_SECRET))
the_strokes_uri = 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'

In [113]:
# # gets artist albums
# results = spotify.artist_albums(the_strokes_uri, album_type='album')
# albums = results['items']
# while results['next']:
#     results = spotify.next(results)
#     albums.extend(results['items'])

In [162]:
results = spotify.artist_top_tracks(the_strokes_uri)

In [121]:
# type(results['tracks'][0])
# results['tracks'][0]

In [194]:
def get_track_statistics(tracks):
    """takes tracks and returns a dictionary of their analysis and features"""
    # track_ids = get_track_ids(tracks)
    dict_ = {}
    # looping through each track and creating nested dictionary where the keys are the track names and the values are the merged track audio analyses and features data
    for track in tqdm(results['tracks']):
        dict_[track['name']] = spotify.audio_features(track['id'])[0] | spotify.audio_analysis(track['id'])['track']
    return dict_

In [215]:
top_tracks = get_track_statistics(results)

  0%|          | 0/10 [00:00<?, ?it/s]

In [216]:
# removing extremely long unnecessary string data
for track in top_tracks:
    del top_tracks[track]['codestring']
    del top_tracks[track]['echoprintstring']
    del top_tracks[track]['synchstring']
    del top_tracks[track]['rhythmstring']
    del top_tracks[track]['sample_md5']

In [229]:
# top_tracks

In [231]:
df = pd.DataFrame(top_tracks).T
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,end_of_fade_in,start_of_fade_out,tempo_confidence,time_signature_confidence,key_confidence,mode_confidence,code_version,echoprint_version,synch_version,rhythm_version
The Adults Are Talking,0.593,0.749,5,-5.67,1,0.0475,0.0113,0.106,0.314,0.649,...,0.38308,306.45117,0.851,0.85,0.771,0.401,3.15,4.12,1.0,1.0
Last Nite,0.624,0.899,0,-5.709,1,0.0296,0.0221,0.000137,0.0975,0.79,...,0.45805,191.33823,0.268,1.0,1.0,0.879,3.15,4.12,1.0,1.0
Reptilia,0.489,0.649,9,-5.11,1,0.0336,0.000603,0.713,0.101,0.77,...,5.41605,219.82666,0.534,0.914,0.223,0.0,3.15,4.12,1.0,1.0
Someday,0.539,0.521,9,-7.46,1,0.0299,0.00425,0,0.367,0.679,...,0.38308,183.44,0.722,1.0,0.684,0.605,3.15,4.12,1.0,1.0
You Only Live Once,0.63,0.908,11,-2.42,1,0.0326,0.0238,0.592,0.116,0.968,...,0.0,177.83583,0.719,1.0,0.875,0.687,3.15,4.12,1.0,1.0
"Call It Fate, Call It Karma",0.544,0.241,4,-14.779,0,0.03,0.983,0.774,0.101,0.359,...,0.0,193.6312,0.229,1.0,0.381,0.59,3.15,4.12,1.0,1.0
Selfless,0.538,0.679,4,-4.297,1,0.0333,0.191,0.00175,0.0889,0.0888,...,0.0,211.4293,0.614,0.847,0.296,0.45,3.15,4.12,1.0,1.0
Under Cover of Darkness,0.35,0.775,11,-3.364,1,0.054,0.00433,0.283,0.0676,0.588,...,0.07669,232.18794,0.087,1.0,0.596,0.671,3.15,4.12,1.0,1.0
12:51,0.451,0.889,4,-3.838,1,0.0379,0.312,0.8,0.558,0.592,...,7.05887,148.3755,1.0,1.0,0.811,0.532,3.15,4.12,1.0,1.0
Hard To Explain,0.408,0.666,7,-6.286,1,0.0578,0.000455,0.862,0.0889,0.666,...,0.38707,224.33333,0.465,1.0,0.717,0.675,3.15,4.12,1.0,1.0


In [228]:
df.to_json('src/top_tracks.json',orient='columns')
df.to_excel('src/top_tracks.xlsx')